In [12]:
import cv2
import os
from PIL import Image
import matplotlib.image as mpimg
import numpy as np
from tqdm import tqdm

def draw_mask(image, mask, color, thresh):
    image = np.require(image, dtype='f4', requirements=['O', 'W'])
    image_c = image.copy()
    
    image[:,:,:][mask[:,:] > thresh] = color
    out_image = 0.5 * image + 0.5 * image_c

    return out_image


# 
main_dir = './CatBack_0129-0206'
gt_src = f'{main_dir}/gtFine/'   # png
img_src = f'{main_dir}/images/'   # jpg
pre_src = f'{main_dir}/PID_pre/'  # jpg
dst = os.path.join(main_dir, dst)
if not os.path.exists(dst):
    os.mkdir(dst)

img_name_l = [i.split('.')[0] for i in os.listdir(img_src)]

for img in tqdm(img_name_l):
    im = mpimg.imread(os.path.join(img_src, f'{img}.jpg'), cv2.IMREAD_COLOR)
    mask_gt = cv2.imread(os.path.join(gt_src, f'{img}.png'), cv2.IMREAD_GRAYSCALE)
    mask_pre = cv2.imread(os.path.join(pre_src, f'{img}.jpg'), cv2.IMREAD_GRAYSCALE)
    im = im[:,:,::-1]

    col_gt = np.array([0.1, 0.1, 0.8])*225  # red
    col_pre= np.array([0.8, 0.1, 0.1])*225  # blue

    im = draw_mask(im, mask=mask_gt, color=col_gt, thresh=100)
    im = draw_mask(im, mask=mask_pre, color=col_pre, thresh=100)

    cv2.imwrite(os.path.join(dst, f'{img}.jpg'), im)



100%|██████████| 513/513 [00:45<00:00, 11.20it/s]


In [45]:
l = []
for i in range(len(mask_pre)):
    for j in range(len(mask_pre[0])):
        if mask_pre[i,j] >0:
            l.append(mask_pre[i,j])
print(set(l))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 39, 217, 218, 219, 220, 221, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255}


In [ ]:


def union_image_mask(image_path, mask_path, num):
    # 读取原图
    image = cv2.imread(image_path)

    # 读取分割mask，这里本数据集中是白色背景黑色mask
    mask_2d = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    # 裁剪到和原图一样大小
    mask_2d = mask_2d[0:400, 0:500]
    h, w = mask_2d.shape
    cv2.imshow("2d", mask_2d)

    # 在OpenCV中，查找轮廓是从黑色背景中查找白色对象，所以要转成黑色背景白色mask
    mask_3d = np.ones((h, w), dtype='uint8')*255
    # mask_3d_color = np.zeros((h,w,3),dtype='uint8')
    mask_3d[mask_2d[:, :] == 255] = 0
    cv2.imshow("3d", mask_3d)
    ret, thresh = cv2.threshold(mask_3d, 127, 255, 0)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    cv2.drawContours(image, [cnt], 0, (0, 255, 0), 1)
    # 打开画了轮廓之后的图像
    cv2.imshow('mask', image)
    # 保存图像
    # cv2.imwrite("./image/result/" + str(num) + ".bmp", image)
